In [1]:
import sys
import os

# Add the root directory to the Python environment
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_dir)

In [2]:
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from src import uniswap, uniprice_2
import warnings
from src.graph import create_graph
from src.algo_per_coin import safe_literal_eval
import json
import re
import concurrent.futures

In [3]:
df = pd.read_csv(os.path.join("../data", 'snapshots', f'snapshot_22816000_full.csv.gz'))
df = df.drop_duplicates('id')

df['token0Price'] = pd.to_numeric(df['token0Price'], errors='coerce')
df['token1Price'] = pd.to_numeric(df['token1Price'], errors='coerce')
df['totalValueLockedUSD'] = pd.to_numeric(df['totalValueLockedUSD'], errors='coerce')
df['other'] = df['other'].apply(safe_literal_eval)

df = df.dropna(subset=['other', 'token0Price', 'token1Price'])

# Check if 'feeTier' column exists, if not, create it
if 'feeTier' not in df.columns:
    df['feeTier'] = df.apply(
        lambda row: int(row['other'].get('feeTier', 0)) if row['version'] in ['v3', 'v4'] 
        else 3000 if row['version'] == 'v2' 
        else None, 
        axis=1
    )
    df['feeTier'] = df['feeTier'].apply(float)

if 'reserve0' not in df.columns:
    df['reserve0'] = pd.to_numeric(df['other'].apply(lambda x: x.get('reserve0', None) if x is not None else None), errors='coerce')

if 'reserve1' not in df.columns:
    df['reserve1'] = pd.to_numeric(df['other'].apply(lambda x: x.get('reserve1', None) if x is not None else None), errors='coerce')

if 'tick' not in df.columns:
    df['tick'] = pd.to_numeric(df['other'].apply(lambda x: x.get('tick', None) if x is not None else None), errors='coerce')

if 'sqrtPrice' not in df.columns:
    df['sqrtPrice'] = pd.to_numeric(df['other'].apply(lambda x: x.get('sqrtPrice', None) if x is not None else None), errors='coerce')

df.set_index('id', inplace=True)
df.head()

/tmp/ipykernel_79498/4038603039.py:1: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("../data", 'snapshots', f'snapshot_22816000_full.csv.gz'))


,createdAtBlockNumber,token0Price,token1Price,liquidity,totalValueLockedUSD,other,token0_decimals,token0_id,token0_name,token0_symbol,...,token1_id,token1_name,token1_symbol,version,block_number,feeTier,reserve0,reserve1,tick,sqrtPrice
id,,,,,,,,,,,,,,,,,,,,,
0x76b36afc7f1ef4bbf3301b602c9f6ca2ff6c3bd8,13844117.0,1.660335e+17,0.000000e+00,934796603.601451,8269.914769,"{'totalSupply': '26538.175287407292876187', 'r...",9.0,0x5d8038644608d1f849ed2c6863a2ea667e53371a,INUGAMI,INUGAMI,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0,3.809035e+17,2.294137e+00,NaN,NaN
0x9fe0f9ef9c17a1afa2c29078ec3323d4cf7a9ddc,17787279.0,2.100182e-09,4.761492e+08,38571.818436,8269.085571,"{'totalSupply': '37947.331922020551982986', 'r...",18.0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,...,0xcd3b1f5b3bad4ef75f2bc840b005b55f7afbe4aa,X Æ A-12 Musk,LilX,v2,22815924.0,3000.0,1.767659e+00,8.416696e+08,NaN,NaN
0x6b8faec9d9e4c5d1b3f74b7640840544d378f6a4,22460490.0,3.441434e+07,2.905765e-08,9317.689528,8268.901696,"{'totalSupply': '0.282842712474618009', 'reser...",9.0,0xbbb4099798b46a82283ef3b1aa3007cf4c17870f,Tomato Frog,TFROG,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0,5.466105e+07,1.588322e+00,NaN,NaN
0x537bb3598a20937c4bfdb413c6d46c13cd0a5dd6,20428614.0,1.838279e+11,5.439800e-12,639462.487731,8268.738322,"{'totalSupply': '18.345353635185121808', 'rese...",9.0,0x3be7bbe6d61eda1524a904e5dde46a22afeb35e8,COCONUT,COCONUT,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0,2.741705e+11,1.491452e+00,NaN,NaN
0xb5b8aee2af5b8d485bac4ed80c71dd8594096dfa,17137011.0,1.430884e+14,6.900000e-15,19508999.450781,8267.930282,"{'totalSupply': '577.061521850140341555', 'res...",9.0,0x80b7a77d280a4ef3365cd34316231b7ee6e8b0c5,Pepe Tate,PATE,...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,v2,22815924.0,3000.0,2.333658e+14,1.630921e+00,NaN,NaN


In [4]:
# df_top = uniswap.get_top_tokens(df, 100000)
# df_top.to_csv(
#     os.path.join("../data", f'top_tokens_22816000.csv'),
#     index=False
# )

In [ ]:
G = create_graph(df, apply_fee=True, remove_low_degree_nodes=False, remove_multi_edges=True, fake_fee=0)
print(f"Graph created with {len(G.nodes())} nodes and {len(G.edges())} edges.")

Graph created with 423870 nodes and 868738 edges.


In [6]:
with open(os.path.join("../data", "graphs", "graph_22816000.pkl"), "wb") as f:
    pickle.dump(G, f)

In [8]:
with open(os.path.join("../data", "graphs", "graph_22816000.pkl"), "rb") as f:
    pickle.load(f)

In [7]:
# df[[
#     'token0_id', 'token1_id',
#     'version', 
#     'feeTier',
#     'token0Price', 'token1Price', 
#     'sqrtPrice', 'tick', # For v3 and v4
#     'reserve0', 'reserve1' # for v2
#     ]].reset_index().to_csv(
#         os.path.join("../data", 'snapshots', f'snapshot_22816000.csv'),
#         index=False
#     )